In [1]:
# !pip install --upgrade langchain openai

# Chains in LangChain

Original Source: [LangChain for LLM Application Development](https://learn.deeplearning.ai/langchain/lesson/4/chains)

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [2]:
from langchain.chains import (
    LLMChain,
    SimpleSequentialChain,
    SequentialChain
)
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate


In [3]:
# OpenAI API KEY
api_key = '?'

In [4]:
llm = ChatOpenAI(openai_api_key=api_key, temperature=0.9)

In [5]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
product = "Artistic and visually stunning hand bags."
result = chain.run(product)
print(result)

"Artistry Bags"


---

## SimpleSequentialChain

In [8]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [9]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 30 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [10]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
)
result = overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Exquisite Creations"
Exquisite Creations offers handcrafted, unique and personalized items for special occasions. With attention to detail and quality, they create memorable pieces that bring joy and elegance to any event.

> Finished chain.


In [11]:
print(result)

Exquisite Creations offers handcrafted, unique and personalized items for special occasions. With attention to detail and quality, they create memorable pieces that bring joy and elegance to any event.


---
## SequentialChain

In [12]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(
    llm=llm, prompt=first_prompt,
    output_key="English_Review"
)

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(
    llm=llm, prompt=second_prompt,
    output_key="summary"
)

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(
    llm=llm, prompt=third_prompt,
    output_key="language"
)

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(
    llm=llm, prompt=fourth_prompt,
    output_key="followup_message"
)


In [13]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=False
)

french_review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

result = overall_chain(french_review)

In [14]:
print(result)

{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?", 'English_Review': "I find the taste mediocre. The foam does not hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?", 'summary': 'The reviewer is disappointed with the mediocre taste and lack of foam retention, questioning whether the product is either an old batch or a counterfeit.', 'followup_message': "Réponse : Nous sommes désolés d'apprendre que notre produit ne répond pas à vos attentes. Nous tenons à vous assurer que nous prenons vos commentaires très au sérieux. Nous maintenons des normes de qualité élevées pour tous nos produits et nous nous engageons à fournir une expérience satisfaisante à nos clients. Nous aimerions en savoir plus sur votre expérience afin de résoudre tout problème éventuel. Pourriez-vous nous fournir plus de détails sur la date

---

## Router Chain

In [15]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [16]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [17]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [18]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [19]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [20]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [21]:
result = chain.run("What is black body radiation?")
print (result)



> Entering new MultiPromptChain chain...


/home/bsantanna/anaconda3/envs/ml-dev/lib/python3.9/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.
Black body radiation refers to the thermal radiation emitted by an object that absorbs all incoming electromagnetic radiation and does not reflect or transmit any of it. It is called "black body" because it absorbs all colors of light, appearing black in color.

When an object is heated, such as a metal rod or even a star, it emits electromagnetic radiation. According to Planck's radiation law, the intensity of this radiation depends on the temperature of the object. A hotter object emits more radiation and at higher frequencies.

The radiation emitted by a black body covers a wide range of frequencies, creating a continuous spectrum. The distribution of this radiation over different frequencies is described by Planck's black body radiation law. It states that the intensity of radiation at different frequencies is given by a specific mathematical formula.

Black body radiation played a crucial role in the development

In [23]:
result = chain.run("What is the most efficient list sorting algorithm?")
print(result)



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the most efficient list sorting algorithm?'}
> Finished chain.
There is no one-size-fits-all answer to this question, as the efficiency of a sorting algorithm depends on various factors such as the size and nature of the input data, available memory, and specific requirements of the application.

However, in terms of general efficiency, the "QuickSort" algorithm is often considered one of the fastest and most efficient sorting algorithms. QuickSort utilizes a divide-and-conquer strategy, where it selects a pivot element and rearranges the elements in the list such that all elements smaller than the pivot are placed to its left, and all elements larger than the pivot are placed to its right. This process is recursively applied to the sublists formed on either side of the pivot until the entire list is sorted.

The average and best-case time complexity of QuickSort is O(n log n), where n is the number of elem